# Process raw data


python -m venv myenv

1️⃣ Kích hoạt venv trong Terminal
myenv\Scripts\activate

2️⃣ Cài thư viện bằng pip trong Terminal

pip install numpy opencv-python tensorflow scikit-learn notebook ipykernel

3️⃣ Đăng ký kernel venv với Jupyter

python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"



In [1]:
import os
import numpy as np
import cv2 as cv

In [2]:
# Path of the directory with raw images
raw_dir = 'archive/train'
folders = ['Open_Eyes', 'Closed_Eyes']

# Path of the directory where processed images will be stored
processed_dir = 'working'


In [3]:
# Mean and std of ImageNet will be used to normalize the images
mean = np.array([0.485, 0.456, 0.406])
std  = np.array([0.229, 0.224, 0.225])

images = []
labels = []

def process_image(img_path):
    img = cv.imread(img_path)
    if img is None:
        print(f"Error loading {img_path}.")
        return None
    # Resize image to 224 x 224 as this image size is expected by ResNet or MobileNet which will be used for image classification
    img = cv.resize(img, (224, 224))
    # cv2 uses the BGR color format, so we convert it to the RGB format
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    # Normalize the image values to range [0, 1]
    img = img / 255.0
    img = (img - mean) / std
    return img


for label, folder in enumerate(folders):
    folder_path = os.path.join(raw_dir, folder)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        img = process_image(file_path)
        if img is not None:
            images.append(img)
            labels.append(label)

# Convert images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Save processed images and labels
np.save(os.path.join(processed_dir, "images.npy"), images)
np.save(os.path.join(processed_dir, "labels.npy"), labels)

In [4]:
from sklearn.model_selection import train_test_split

# Load the processed dataset
images = np.load(os.path.join(processed_dir, "images.npy"))
labels = np.load(os.path.join(processed_dir, "labels.npy"))

'''Split the dataset into train+val (80%) and test datasets (20%). The data is automatically shuffled. 
stratify=labels: the generated splits gave the same proprotion of labels as given by parameter "labels".'''
images_train_val, images_test, labels_train_val, labels_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Split the train_val dataset into train (75%) and val datasets (25%)
images_train, images_val, labels_train, labels_val = train_test_split(images_train_val, labels_train_val, test_size=0.25, random_state=42, stratify=labels_train_val)

print(f"Train set size: {images_train.shape[0]}")
print(f"Validation set size: {images_val.shape[0]}")
print(f"Test set size: {images_test.shape[0]}")

# Save split data and labels
np.save(os.path.join(processed_dir, "train_images.npy"), images_train)
np.save(os.path.join(processed_dir, "train_labels.npy"), labels_train)
np.save(os.path.join(processed_dir, "val_images.npy"), images_val)
np.save(os.path.join(processed_dir, "val_labels.npy"), labels_val)
np.save(os.path.join(processed_dir, "test_images.npy"), images_test)
np.save(os.path.join(processed_dir, "test_labels.npy"), labels_test)

Train set size: 2400
Validation set size: 800
Test set size: 800


# Finetune ResNet50

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers, Sequential

def get_resnet50_model(variant='base'):
    # Load the ResNet50 model pretrained on ImageNet
    base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
    train_from = 150

    if variant == 'finetune':
        base_model.trainable = True
        for layer in base_model.layers[:train_from]:
            layer.trainable = False
    else:
        # Freeze all layers
        base_model.trainable = False
    
    model_append = [layers.GlobalAveragePooling2D()]
    model_append.append(layers.Dense(1, activation='sigmoid'))
    model = Sequential([base_model] + model_append)

    return model

# Train

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC, F1Score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Load train dataset
X_train = np.load(os.path.join(processed_dir, "train_images.npy"))
y_train = np.load(os.path.join(processed_dir, "train_labels.npy"))
# Reshape y_train (num_train_examples,) to have the same shape as model predictions in tf (num_train_examples, 1)
y_train = y_train.reshape(-1, 1)

# Load validation dataset
X_val = np.load(os.path.join(processed_dir, "val_images.npy"))
y_val = np.load(os.path.join(processed_dir, "val_labels.npy"))
# Reshape y_val (num_val_examples,) to have the same shape as model predictions in tf (num_val_examples, 1)
y_val = y_val.reshape(-1, 1)

# Choose model variant: 'base', 'finetune'
model_variant = 'finetune'
# Load ResNet50 model
model = get_resnet50_model(variant=model_variant)

# Configure model settings for training
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryCrossentropy(), metrics=[BinaryAccuracy(), Precision(), Recall(), AUC(), F1Score(threshold=0.5)])

# Callbacks
# Create the EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(f"working/{model_variant}.h5", save_best_only=True)

# Train modified ResNet50
history = model.fit(X_train, y_train, validation_data=[X_val, y_val], batch_size=32, epochs=15, callbacks=[early_stop, checkpoint])

Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - auc: 0.9601 - binary_accuracy: 0.8942 - f1_score: 0.8988 - loss: 0.2075 - precision: 0.8772 - recall: 0.9261

75/75 ━━━━━━━━━━━━━━━━━━━━ 857s 11s/step - auc: 0.9944 - binary_accuracy: 0.9579 - f1_score: 0.9581 - loss: 0.0963 - precision: 0.9530 - recall: 0.9633 - val_auc: 0.9976 - val_binary_accuracy: 0.5213 - val_f1_score: 0.0815 - val_loss: 0.5608 - val_precision: 1.0000 - val_recall: 0.0425
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - auc: 0.9995 - binary_accuracy: 0.9861 - f1_score: 0.9852 - loss: 0.0306 - precision: 0.9810 - recall: 0.9900

75/75 ━━━━━━━━━━━━━━━━━━━━ 820s 11s/step - auc: 0.9996 - binary_accuracy: 0.9875 - f1_score: 0.9875 - loss: 0.0285 - precision: 0.9875 - recall: 0.9875 - val_auc: 0.9992 - val_binary_accuracy: 0.8400 - val_f1_score: 0.8095 - val_loss: 0.3200 - val_precision: 1.0000 - val_recall: 0.6800
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 663s 9s/step - auc: 0.9995 - binary_accuracy: 0.9975 - f1_score: 0.9975 - loss: 0.0112 - precision: 0.9967 - recall: 0.9983 - val_auc: 0.9875 - val_binary_accuracy: 0.7075 - val_f1_score: 0.5866 - val_loss: 0.6838 - val_precision: 1.0000 - val_recall: 0.4150
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - auc: 1.0000 - binary_accuracy: 0.9977 - f1_score: 0.9977 - loss: 0.0070 - precision: 0.9973 - recall: 0.9981

75/75 ━━━━━━━━━━━━━━━━━━━━ 615s 8s/step - auc: 0.9999 - binary_accuracy: 0.9967 - f1_score: 0.9967 - loss: 0.0111 - precision: 0.9958 - recall: 0.9975 - val_auc: 0.9975 - val_binary_accuracy: 0.9312 - val_f1_score: 0.9357 - val_loss: 0.1612 - val_precision: 0.8791 - val_recall: 1.0000
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 789s 11s/step - auc: 1.0000 - binary_accuracy: 0.9996 - f1_score: 0.9996 - loss: 0.0040 - precision: 0.9992 - recall: 1.0000 - val_auc: 0.9187 - val_binary_accuracy: 0.6562 - val_f1_score: 0.4762 - val_loss: 1.3627 - val_precision: 1.0000 - val_recall: 0.3125
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - auc: 1.0000 - binary_accuracy: 1.0000 - f1_score: 1.0000 - loss: 6.9346e-04 - precision: 1.0000 - recall: 1.0000

75/75 ━━━━━━━━━━━━━━━━━━━━ 902s 12s/step - auc: 1.0000 - binary_accuracy: 1.0000 - f1_score: 1.0000 - loss: 5.8723e-04 - precision: 1.0000 - recall: 1.0000 - val_auc: 1.0000 - val_binary_accuracy: 0.9950 - val_f1_score: 0.9950 - val_loss: 0.0189 - val_precision: 1.0000 - val_recall: 0.9900
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - auc: 1.0000 - binary_accuracy: 1.0000 - f1_score: 1.0000 - loss: 3.6862e-04 - precision: 1.0000 - recall: 1.0000

75/75 ━━━━━━━━━━━━━━━━━━━━ 837s 11s/step - auc: 1.0000 - binary_accuracy: 1.0000 - f1_score: 1.0000 - loss: 4.0995e-04 - precision: 1.0000 - recall: 1.0000 - val_auc: 1.0000 - val_binary_accuracy: 0.9975 - val_f1_score: 0.9975 - val_loss: 0.0088 - val_precision: 0.9950 - val_recall: 1.0000
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 739s 10s/step - auc: 0.9997 - binary_accuracy: 0.9942 - f1_score: 0.9942 - loss: 0.0200 - precision: 0.9933 - recall: 0.9950 - val_auc: 0.9987 - val_binary_accuracy: 0.9937 - val_f1_score: 0.9937 - val_loss: 0.0220 - val_precision: 0.9950 - val_recall: 0.9925
Epoch 9/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - auc: 1.0000 - binary_accuracy: 0.9990 - f1_score: 0.9990 - loss: 0.0044 - precision: 0.9981 - recall: 1.0000

75/75 ━━━━━━━━━━━━━━━━━━━━ 695s 9s/step - auc: 1.0000 - binary_accuracy: 0.9996 - f1_score: 0.9996 - loss: 0.0031 - precision: 0.9992 - recall: 1.0000 - val_auc: 1.0000 - val_binary_accuracy: 0.9975 - val_f1_score: 0.9975 - val_loss: 0.0051 - val_precision: 1.0000 - val_recall: 0.9950
Epoch 10/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 757s 10s/step - auc: 1.0000 - binary_accuracy: 1.0000 - f1_score: 1.0000 - loss: 0.0015 - precision: 1.0000 - recall: 1.0000 - val_auc: 0.9975 - val_binary_accuracy: 0.9550 - val_f1_score: 0.9569 - val_loss: 0.1155 - val_precision: 0.9174 - val_recall: 1.0000
Epoch 11/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 756s 10s/step - auc: 1.0000 - binary_accuracy: 1.0000 - f1_score: 1.0000 - loss: 3.0787e-04 - precision: 1.0000 - recall: 1.0000 - val_auc: 1.0000 - val_binary_accuracy: 0.9975 - val_f1_score: 0.9975 - val_loss: 0.0117 - val_precision: 0.9950 - val_recall: 1.0000
Epoch 12/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 718s 10s/step - auc: 1.0000 - binary_accuracy: 1.0000 - f1_score: 1.0000 

# Test

In [8]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

# Load test dataset
X_test = np.load("working/test_images.npy")
y_test = np.load("working/test_labels.npy")
# Reshape y_test (num_test_examples,) to have the same shape as model predictions in tf (num_test_examples, 1)
y_test = y_test.reshape(-1, 1)

# Load the updated resnet model
model = load_model("working/finetune.h5")

# Evaluate on the test dataset
results = model.evaluate(X_test, y_test, batch_size=32, verbose=1)

# Predict labels
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Open", "Closed"]))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

25/25 ━━━━━━━━━━━━━━━━━━━━ 181s 7s/step - auc: 1.0000 - binary_accuracy: 0.9987 - f1_score: 0.9987 - loss: 0.0054 - precision: 1.0000 - recall: 0.9975
25/25 ━━━━━━━━━━━━━━━━━━━━ 180s 7s/step

Classification Report:
              precision    recall  f1-score   support

        Open       1.00      1.00      1.00       400
      Closed       1.00      1.00      1.00       400

    accuracy                           1.00       800
   macro avg       1.00      1.00      1.00       800
weighted avg       1.00      1.00      1.00       800


Confusion Matrix:
[[400   0]
 [  1 399]]
